In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]='PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"]='1'

import tensorflow as tf
import keras
from keras import layers
from matplotlib import pyplot as plt
from keras.datasets import cifar10
import glob
import numpy as np
import cv2
from PIL import Image
import random
path = glob.glob("/notebooks/Test_Env/STAT_PRED/1ch/12x12/*.png")
print("total: "+str(len(path)))

path.sort()
print(path[0])

temp = []
label_temp = []

for ddd in range(0,9):
    label_temp_inner = []
    label_temp.append(label_temp_inner)

for i,n in enumerate(path):
    #temp.append(cv2.imread(n))
    pict_t = cv2.imread(n,0)
    label_temp_inner = []
    
    gray = np.ndarray((pict_t.shape[0],pict_t.shape[1],1),dtype = int)
    gray = pict_t.reshape((pict_t.shape[0],pict_t.shape[1],1))
    
    temp.append(gray)
    noname = n[n.rindex("/"):]
    noname = noname[noname.index("_")+1:]
    
    noname = noname[:noname.rindex(".png")]
    k = noname.split("_")
    
   
    for idx in range(0,9):
        r = np.ndarray((1,))
        r[0] = float(k[idx])
        label_temp[idx].append(r)
pic = np.array(temp)
label = np.array(label_temp)
      
maxlist=[]
for i in label:
    maxlist.append(np.max(i))

for i in range (len(label)):
    label[i]/=maxlist[i]
    
pic = pic / 255.0


Using TensorFlow backend.


total: 849
/notebooks/Test_Env/STAT_PRED/1ch/12x12/LIDC-IDRI-0001_5_1_6_3.5_3.5_3_4.5_4.75_4.75.png


In [4]:
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Flatten
from tensorflow.keras.models import Model
def get_model_x(input_shape):
    
    
    inputs = Input(input_shape)

    X = Conv2D(filters=32, kernel_size=(5, 5),
               activation='relu', padding='same')(inputs)
    X = BatchNormalization()(X)
    X = MaxPooling2D(pool_size=(2, 2))(X)

    X = Conv2D(filters=64, kernel_size=(3, 3),
               activation='relu', padding='same')(X)
    X = BatchNormalization()(X)
    X = MaxPooling2D(pool_size=(2, 2))(X)

    X = Conv2D(filters=128, kernel_size=(3, 3),
               activation='relu', padding='same')(X)
    X = BatchNormalization()(X)
    X = MaxPooling2D(pool_size=(2, 2))(X)
    
    X = Conv2D(filters=256, kernel_size=(3, 3),
               activation='relu', padding='same')(X)

    X = Flatten()(X)

    X = Dense(1024, activation='sigmoid')(X)
    X = Dense(512, activation='sigmoid')(X)
    X = Dense(1, activation='sigmoid')(X)
    #,kernel_initializer="he_normal"
    model = Model(inputs=[inputs], outputs=[X])
    return model

def get_adaptive_lr_callback():
    from tensorflow.keras.callbacks import LearningRateScheduler

    def lr_scheduler(epoch, lr):
        decay_rate = 0.1
        decay_step = 50
        if epoch % decay_step == 0 and epoch > decay_step:
            return lr * decay_rate
        return lr

    adaptive_lr_callback = LearningRateScheduler(lr_scheduler)
    return adaptive_lr_callback

def compile_model(model, lr):
    from tensorflow.keras.optimizers import Adam, RMSprop
    from tensorflow import keras
    optimizer = Adam(lr=lr)
    model.compile(optimizer=optimizer,
                  loss=keras.losses.mean_squared_error,
                  metrics=[
                      keras.metrics.mean_absolute_error,
                      keras.metrics.mean_squared_error
                  ])
    return model

In [3]:
#save evaluation txt

def save_eval(i,h,model,name = "temp.txt"):
    fs = open('./'+name,'a')
    fs.write(str(i+1)+" "+str(min(h.history['val_mean_absolute_error'])*maxlist[i])+'\n')
    print('finished: '+str(i+1))
        


In [ ]:
# train and save model 
import gc
save_path = './final_model2'
import pickle
# 5.0, 4.0, 6.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0
namedict=['place_holder',
        'Subtlety',
        'Internal Structure',
         'Calcification',
         'Sphericity',
         'Margin',
         'Lobulation',
         'Spiculation',
         'Texture',
         'Malignancy']

for i in range(0,9):
    
    mcp_save = keras.callbacks.ModelCheckpoint(save_path+'/'+namedict[i+1]+'_1ch_12x12.hdf5', save_best_only=True,verbose = 0, monitor='val_mean_absolute_error', mode='min')
    
    
    #print(pic.shape)
    split=int(len(pic)*0.80)
    print("start: "+str(i+1))
    h_plot = []
        
    model = get_model_x((128,128,1))
    model = compile_model(model,5e-5)
    
    h=model.fit(pic,label[i],validation_split = 0.2 ,batch_size=32,epochs=1,verbose=0,callbacks=[
            get_adaptive_lr_callback(),
            mcp_save
    ])
    save_eval(i,h,model,'all_train.txt')
    del model
    gc.collect()

In [5]:
# train and test by fold
namedict=['Subtlety',
        'Internal Structure',
         'Calcification',
         'Sphericity',
         'Margin',
         'Lobulation',
         'Spiculation',
         'Texture',
         'Malignancy']
import gc
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf.get_n_splits(len(pic))

result = {}
for i in range(0,1):
    #9 stats
    print(namedict[i])
    result[namedict[i]] = {}
    for fold_id, (train_index, test_index) in enumerate(kf.split(pic), start=1):
        # by fold
        
        print(fold_id)
        #print(train_index[0],train_index[-1], test_index[0],test_index[-1] )
        model = get_model_x((128,128,1))
        model = compile_model(model,5e-5)
    
        h=model.fit(pic[train_index],label[i][train_index],
                    validation_data=(pic[test_index],label[i][test_index]),
                    batch_size=30,
                    epochs=150,
                    verbose=0,
                    callbacks=[
            get_adaptive_lr_callback()
        ])
        result[namedict[i]]['fold'+str(fold_id)] = min(h.history['val_mean_absolute_error'])*maxlist[i]
        del model
        gc.collect()

Subtlety
1
2
3
4
5


In [6]:
import json
with open('result_fold.json','w') as jjj:
    json.dump(result,jjj)